In [2]:
import pandas as pd
import numpy as np
import os
from tabulate import tabulate



In [29]:
def _readReport():
    global GlobalVar
    GlobalVar.reportDf = pd.read_excel(GlobalVar.fileName, header = 0)
    #replace datetime to only date 
    for dateColumn in GlobalVar.dateMetadata:
        GlobalVar.reportDf[dateColumn] = GlobalVar.reportDf[dateColumn].dt.date
    #set all cell dataType of dataframe to str
    GlobalVar.reportDf = GlobalVar.reportDf.astype(str)
    #replace nan to empty string
    GlobalVar.reportDf = GlobalVar.reportDf.replace(np.nan, "")
    _reorder()
    
def _reorder():
    global GlobalVar
    checkPart = GlobalVar.reportDf.loc[GlobalVar.reportDf['SKILL'] == "Check"].sort_values(by=["AP"])
    complementaryPart = GlobalVar.reportDf.loc[GlobalVar.reportDf['SKILL'] != "Check"].sort_values(by=["OA_NO", "AP", "OA_DESC", "SKILL"])
    if checkPart.size > 0:
        GlobalVar.reportDf = pd.concat([checkPart, complementaryPart]).reset_index()
    else:
        GlobalVar.reportDf = complementaryPart.reset_index()
#         complementaryPart = GlobalVar.reportDf.loc[GlobalVar.reportDf['SKILL'] != "Check"]
#         complementaryPart = complementaryPart.sort_values(by=["OA_NO", "AP", "SKILL"])
    

def _showBrief():
    global GlobalVar
    displayMetadata = list()
    for i in range(len(GlobalVar.displayColumns)):
        displayMetadata.append(GlobalVar.metadata[GlobalVar.displayColumns[i]])
    print(tabulate(GlobalVar.reportDf[displayMetadata], headers='keys', tablefmt='psql'))

def _saveXlsx():
    global GlobalVar
    GlobalVar.reportDf.to_excel(GlobalVar.fileName, index = False)
    
def _controller():
    global GlobalVar
    showFlag = True
    action = None
    _showBrief()
    while(True):
        action = input("To do? ").lower().strip()
        if action in GlobalVar.functionDict:
#             try:
                GlobalVar.functionDict[action]()
#             except:
#                 print("Unexpected error:", sys.exc_info()[0])
        elif action == "?":
            for key in GlobalVar.functionDict:
                print(key)
        elif action == "ex":
            break
        else:
            print("Unknow function")
            

def displayAll():
    global GlobalVar
    print(tabulate(GlobalVar.reportDf, headers='keys', tablefmt='psql'))
    
def addNewRow():
    global GlobalVar
    newDataDict = dict()
    for key in GlobalVar.metadata:
        if key not in GlobalVar.constMetadata:
            newDataDict[key] = input(key + ": ")
        else:
            newDataDict[key] = ""
    GlobalVar.reportDf = GlobalVar.reportDf.append(newDataDict, ignore_index=True)
    _reorder()
    _showBrief()

def saveExcel():
    _saveXlsx()

def editRow():
    global GlobalVar
    index = input("Which row?")
    count = 0
    metadataPair = {"all": "all"}
    for column in GlobalVar.metadata:
        if column not in GlobalVar.constMetadata:
            count += 1
            print(f"{count}.{column}", end = "｜")
            metadataPair[str(count)] = column
    
    while(True):
        editTarget = input("Which column? Or all?").lower().strip()
        if editTarget in metadataPair:
            break
        else:
            print("Not correct. Try again.")
    
    if editTarget != "all":
        GlobalVar.reportDf.at[int(index), metadataPair[editTarget]] = input(f"{GlobalVar.reportDf.at[int(index), metadataPair[editTarget]]} ->")
    else:
        for metadata in GlobalVar.metadata:
            if metadata not in GlobalVar.constMetadata:
                GlobalVar.reportDf.at[int(index), metadata] = input(f"{GlobalVar.reportDf.at[int(index), metadata]} ->")
    _reorder()
    _showBrief()
    
def removeRow():
    global GlobalVar
    index = int(input("Which row?"))
    if input(f"Confirm to delete {index} row? enter n to stop ") != "n":
        GlobalVar.reportDf = GlobalVar.reportDf.drop(GlobalVar.reportDf.index[index])
        _reorder()
        _showBrief()
    else:
        print("canceled")

def _selfCheck():
    if not os.path.isfile("person.congif"):
        firstExecute()

def firstExecute():
    global GlobalVar
    print("First time execute, setting...")
    
    newConfig = open("person.config", "w")
    
    newConfig.write("OWNER=" + input("Owner Name? ").strip() + "\n")
    
    newConfig.write("fileName=" + input("import weekly report? (keep empty if no) file name: ") + "\n")
    
    newConfig.write("simpleDisplatColumn=")
    for index in GlobalVar.displayColumns:
        newConfig.write(GlobalVar.metadata[index] + ", ")
    newConfig.write("\n")


class GlobalVar():
    reportDf = None
    metadata =  ['A_DATE', 'ITEM', 'OA_DESC', 'AP', 'SKILL', 'SITE', 'DUE_DATE', 'COMPLET_D', 'OWNER', 'IT_STATUS', 'OA_NO', 'PROGRAM', 'W_HOUR', 'REMARK', 'PROG_CNT', 'OA_STATUS']
    constMetadata = ['A_DATE', 'ITEM', 'OWNER']
    dateMetadata = ['DUE_DATE', 'COMPLET_D']
    displayColumns = [2, 12, 13]
    functionDict = {"new": addNewRow, "all": displayAll, "save": saveExcel, "edit": editRow, "remove": removeRow}
    #     fileName = "WeeklyReport-V1.0-2019.09.30-TonyOu.xlsx"
    fileName = "new.xlsx"
    
def initializeApp():
    _selfCheck()
    _readReport()
    _controller()


In [30]:
initializeApp()


First time execute, setting...
Owner Name? Tony
import weekly report? (keep empty if no) file name: 
+----+---------------------------------------+----------+---------------------------+
|    | OA_DESC                               |   W_HOUR | REMARK                    |
|----+---------------------------------------+----------+---------------------------|
|  0 | 會議室系統開會                        |      1   | 與Zoe和Ian開會            |
|  1 | 二部臨時部會                          |      0.5 | nan                       |
|  2 | QC七手法報告製作                      |      1   | nan                       |
|  3 | ABAP研究與操作練習                    |      7   | 藉由過去ABAP習題練習      |
|  4 | ABAP上課                              |      3.5 | 由Jeff上課                |
|  5 | MRP SAP 重要參數介紹(MRP 1,2,3,4)上課 |      1.5 | nan                       |
|  6 | [32 Bits 轉64 Bits]會議室系統-公務車  |     25.5 | 1. 研究原始程式碼        |
|    |                                       |          | 2. 與Ian和Zoe討論設計內容 |
|    |                        

In [25]:
file = open(".config", "w")
file.write("test\n")
file.write("test")
file.close()
testList = ", ".join(GlobalVar.constMetadata).split(',')
testList = list(map(str.strip , testList))
print(testList)
print(os.path.isfile(".config"))

['A_DATE', 'ITEM', 'OWNER']
True


NameError: name 'dataclass' is not defined